# 1. Import the additional Libs

In [3]:
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup
import time
import pyodbc
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

# 2. Defining the standard Site Adresses

In [29]:
# Defining the header for the web get
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
headers = {'User-Agent' : user_agent}

#year of test
year_actual = 2022
website_base = "https://fbref.com/"
website_table_results = website_base + "en/comps/22/"+str(year_actual)+"/"+str(year_actual)+"/Major-League-Soccer-Stats"

In [35]:
website_table_results

'https://fbref.com/en/comps/22/2022/2022/Major-League-Soccer-Stats'

# 3. Creating Function to Extract the Data from the site

In [30]:
# Function to get content from website
def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()
    
    except:
        pass


In [6]:
# Function to get Ranking Table from website
def ConsultaWebB_SeasonDetail(rawFullTable_Year):
    
    # Reading the table result to get the link for each team in each conference (east/west)
    soup = BeautifulSoup(rawFullTable_Year)
    standings_table_east = soup.select('table.stats_table')[0]
    standings_table_west = soup.select('table.stats_table')[2]

    # Get the links for East Conference
    links_east = standings_table_east.find_all('a')
    links_east = [l.get("href") for l in links_east]
    links_east = [l for l in links_east if '/squads/' in l]

    # Get the links for West Conference
    links_west = standings_table_west.find_all('a')
    links_west = [l.get("href") for l in links_west]
    links_west = [l for l in links_west if '/squads/' in l]
    
    # Get each club link on the table
    links = links_west + links_east
    team_urls = [f"https://fbref.com{l}" for l in links]
        
    return team_urls

In [7]:
# Function to get Ranking Table from website
def ConsultaWebB_RkTable_links(year):
    
    #Define year and web address
    year_actual = year
    website_base = "https://fbref.com/"
    website_table_results = website_base + "en/comps/22/"+str(year_actual)+"/"+str(year_actual)+"/Major-League-Soccer-Stats"
    
    #Request
    raw_data = ConsultaWebB(website_table_results)
    
    # Getting the Table of Results
    FullTable_EastConf = pd.read_html(raw_data)[0]
    FullTable_WestConf = pd.read_html(raw_data)[2]
    
    ################################ CONFERENCE
    FullTable_EastConf['Conference'] = 'EastConf'
    FullTable_WestConf['Conference'] = 'WestConf'
    
    FullTable_Year = pd.concat([FullTable_EastConf,FullTable_WestConf])
    FullTable_Year["Season"] = year_actual
    
    #using the function to get urls
    team_urls = ConsultaWebB_SeasonDetail(raw_data)
    
    return FullTable_Year, team_urls

In [22]:
# Function to get Ranking Table from website
def ConsultaWebB_RkTable_details(links, actual_year):

    # Creating a DF from details of each team
    match_details = pd.DataFrame()
    for link in links:
        raw_data = ConsultaWebB(link)
        matches_det = pd.read_html(raw_data, match="Scores & Fixtures")[0]
        matches_det["Season"] = actual_year
        matches_det["Team"] = link.split('/')[-1][:-6]

        match_details = pd.concat([match_details,matches_det])

        time.sleep(1)
        
    match_details = match_details[match_details['Round']=='Regular Season']
    
    return match_details

# 4. Creating SQL Functions to Query, Insert and Truncate tables

In [9]:
# Function Select
def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'MLS_Project')
    out = pd.read_sql_query(query,conn)
    return out

query_all_rank = '''
    Select 
    * 
    from TableRankBase
'''
query_all_details = '''
    Select 
    * 
    from TableDetailsBase
'''
query_max_rank = '''
    SELECT
        maior = max(Proposicao)
    from TableDetailsBase
'''
query_max_details = '''
    SELECT
        maior = max(Proposicao)
    from TableDetailsBase
'''


In [10]:
# Function to truncate a table
def SQLTruncate(TableName):
    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    cursor.execute(f'''
    
                   TRUNCATE TABLE {TableName}
                     
                   ''')
    conn.commit()
    cursor.close()

In [11]:
# Create a function to INSERT Table of Ranking
def SQLInsertRanking(TableRanking):
    base = pd.DataFrame(columns=['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP',
                                 'xG', 'xGA', 'xGD', 'xGD/90', 'Attendance', 'Top Team Scorer',
                                 'Goalkeeper', 'Notes', 'Conference', 'Season'])
    TableBase = base.append(TableRanking).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    for index, row in TableBase.iterrows():

        cursor.execute('''

            INSERT INTO TableRankBase (
                Rk, 
                TEAM_NAME, 
                MP, 
                W, 
                D, 
                L, 
                GF, 
                GA, 
                GD, 
                PTS, 
                PTS_MP,
                XG, 
                XGA, 
                XGD, 
                XGD_90, 
                NOTES,
                CONFERENCE,
                SEASON

            )
            Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', # mesma qtde de variaveis

            row['Rk'],
            row['Squad'],
            row['MP'],
            row['W'],
            row['D'],
            row['L'],
            row['GF'],
            row['GA'],
            row['GD'],
            row['Pts'],           
            row['Pts/MP'],           
            row['xG'],           
            row['xGA'],           
            row['xGD'],           
            row['xGD/90'],           
            row['Notes'],           
            row['Conference'], 
            row['Season']
        )

    conn.commit()
    cursor.close()

In [12]:
# Create a function to INSERT Table of Details
def SQLInsertGameDetails(TableGameDetails):
    base = pd.DataFrame(columns=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
                                 'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation',
                                 'Referee', 'Match Report', 'Notes', 'Season', 'Team'])
    TableBase = base.append(TableGameDetails).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    for index, row in TableBase.iterrows():

        cursor.execute('''

            INSERT INTO TableDetailsBase ( 
                TEAM_NAME, 
                GAME_Date,
                GAME_Time,
                Comp,
                SEASON_Round,
                WEEK_DAY,
                TEAM_Venue,
                GAME_Result,
                GF,
                GA,
                Opponent,
                xG,
                xGA,
                Poss,
                Attendance,
                Captain,
                Formation,
                Referee,
                Notes,
                Season

            )
            Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', # mesma qtde de variaveis

            row['Team'],
            row['Date'],
            row['Time'],
            row['Comp'],
            row['Round'],
            row['Day'],
            row['Venue'],
            row['Result'],
            row['GF'],
            row['GA'],           
            row['Opponent'],           
            row['xG'],           
            row['xGA'],           
            row['Poss'],           
            row['Attendance'],           
            row['Captain'],           
            row['Formation'],
            row['Referee'],
            row['Notes'], 
            row['Season']
        )

    conn.commit()
    cursor.close()

# 5. Function GetInsert toguether all Function

In [13]:
def GetInsertAllData(year):
    
    #Getting data from year (Ranking table and details link)
    FullTable_Year_df, team_urls_list = ConsultaWebB_RkTable_links(year)
    
    #Insert data into SQL - Table Rank
    SQLInsertRanking(FullTable_Year_df)
    print('Total of ' + str(len(FullTable_Year_df)) + ' inserted on the database Rank.')
    
    #Getting Teams season details
    FullTable_Year_details = ConsultaWebB_RkTable_details(team_urls_list, year)
    
    #Insert data into SQL - Table Details
    SQLInsertGameDetails(FullTable_Year_details)
    print('Total of ' + str(len(FullTable_Year_details)) + ' inserted on the database Season details.')

# 6. Testing

In [14]:
SQLTruncate('TableDetailsBase')
SQLTruncate('TableRankBase')

In [18]:
GetInsertAllData(2022)

Total of 28 inserted on the database Rank.
Total of 952 inserted on the database Season details.


In [19]:
table_rank = SQLSelect(query_all_rank)
table_rank_details = SQLSelect(query_all_details)

In [47]:
table_rank

,Generalkey,Rk,TEAM_NAME,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,XG,XGA,XGD,XGD_90,NOTES,CONFERENCE,SEASON
0,1,1,Philadelphia,23,14,5,4,44,20,24,47,2.04,35.0,23.5,11.4,0.57,? First Round,EastConf,2020
1,2,2,Toronto FC,23,13,5,5,33,26,7,44,1.91,31.1,22.5,8.6,0.43,? First Round,EastConf,2020
2,3,3,Columbus Crew,23,12,5,6,36,21,15,41,1.78,28.6,28.1,0.5,0.03,? First Round,EastConf,2020
3,4,4,Orlando City,23,11,8,4,40,25,15,41,1.78,27.5,24.3,3.2,0.16,? First Round,EastConf,2020
4,5,5,NYCFC,23,12,3,8,37,25,12,39,1.70,33.3,26.7,6.7,0.33,? First Round,EastConf,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,186,8,Vancouver,34,13,8,13,54,67,-13,47,1.38,52.8,60.3,-7.5,-0.22,,WestConf,2018
186,187,9,Houston Dynamo,34,10,8,16,58,58,0,38,1.12,56.8,55.3,1.6,0.05,,WestConf,2018
187,188,10,Minnesota Utd,34,11,3,20,49,71,-22,36,1.06,48.1,61.4,-13.2,-0.39,,WestConf,2018
188,189,11,Colorado Rapids,34,8,7,19,36,63,-27,31,0.91,40.4,61.4,-21.0,-0.62,,WestConf,2018


In [21]:
table_rank_details

,Generalkey,TEAM_NAME,GAME_Date,GAME_Time,Comp,SEASON_Round,WEEK_DAY,TEAM_Venue,GAME_Result,GF,...,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Notes,Season
0,1,Los-Angeles-FC,2022-02-26,12:30:00,MLS,Regular Season,Sat,Home,W,3,...,Colorado Rapids,2.4,0.4,51,22010.0,Carlos Vela,4-3-3,Armando Villarreal,,2022
1,2,Los-Angeles-FC,2022-03-06,19:00:00,MLS,Regular Season,Sun,Home,D,1,...,Portland Timbers,2.2,0.6,69,22102.0,Carlos Vela,4-3-3,Ismail Elfath,,2022
2,3,Los-Angeles-FC,2022-03-12,13:30:00,MLS,Regular Season,Sat,Away,W,2,...,Inter Miami,0.5,0.4,53,11465.0,Carlos Vela,4-3-3,Ted Unkel,,2022
3,4,Los-Angeles-FC,2022-03-20,19:00:00,MLS,Regular Season,Sun,Home,W,3,...,Vancouver,1.7,1.1,62,22065.0,Carlos Vela,4-3-3,Guido Gonzalez,,2022
4,5,Los-Angeles-FC,2022-04-02,19:30:00,MLS,Regular Season,Sat,Away,W,4,...,Orlando City,2.7,1.4,39,16753.0,Carlos Vela,4-3-3,Joe Dickerson,,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,948,DC-United,2022-09-10,19:30:00,MLS,Regular Season,Sat,Away,D,0,...,Real Salt Lake,0.6,0.5,44,20487.0,Victor Pálsson,3-5-2,Kevin Broadley,,2022
948,949,DC-United,2022-09-13,19:30:00,MLS,Regular Season,Tue,Away,L,0,...,Sporting KC,0.4,1.9,41,15106.0,Steve Birnbaum,4-4-2,R. Mendoza,,2022
949,950,DC-United,2022-09-18,17:00:00,MLS,Regular Season,Sun,Home,L,2,...,Inter Miami,0.8,1.2,43,14284.0,Steve Birnbaum,4-2-3-1,Ismail Elfath,,2022
950,951,DC-United,2022-10-01,19:30:00,MLS,Regular Season,Sat,Away,L,0,...,CF Montréal,0.6,2.0,36,19619.0,Steve Birnbaum,4-2-3-1,Ramy Touchan,,2022


# 7. Inserting Data from 2015 to 2022

In [ ]:
# Defining inital and end year
first_year = 2015
last_year = 2022

#Starting process
print("###################### Process started at ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
# Truncate the tables
SQLTruncate('TableDetailsBase')
SQLTruncate('TableRankBase')

# Running the function for each year
for i in list(range(first_year, last_year+1)):
    print('Starting get data from year: ',i,' at', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    try:
        GetInsertAllData(i)
        time.sleep(2)
        
    except:
        pass
print("###################### Insert all done at", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

In [44]:
table_rank = SQLSelect(query_all_rank)
table_rank_details = SQLSelect(query_all_details)

In [45]:
table_rank

,Generalkey,Rk,TEAM_NAME,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,XG,XGA,XGD,XGD_90,NOTES,CONFERENCE,SEASON
0,1,1,Philadelphia,23,14,5,4,44,20,24,47,2.04,35.0,23.5,11.4,0.57,? First Round,EastConf,2020
1,2,2,Toronto FC,23,13,5,5,33,26,7,44,1.91,31.1,22.5,8.6,0.43,? First Round,EastConf,2020
2,3,3,Columbus Crew,23,12,5,6,36,21,15,41,1.78,28.6,28.1,0.5,0.03,? First Round,EastConf,2020
3,4,4,Orlando City,23,11,8,4,40,25,15,41,1.78,27.5,24.3,3.2,0.16,? First Round,EastConf,2020
4,5,5,NYCFC,23,12,3,8,37,25,12,39,1.70,33.3,26.7,6.7,0.33,? First Round,EastConf,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,186,8,Vancouver,34,13,8,13,54,67,-13,47,1.38,52.8,60.3,-7.5,-0.22,,WestConf,2018
186,187,9,Houston Dynamo,34,10,8,16,58,58,0,38,1.12,56.8,55.3,1.6,0.05,,WestConf,2018
187,188,10,Minnesota Utd,34,11,3,20,49,71,-22,36,1.06,48.1,61.4,-13.2,-0.39,,WestConf,2018
188,189,11,Colorado Rapids,34,8,7,19,36,63,-27,31,0.91,40.4,61.4,-21.0,-0.62,,WestConf,2018


In [46]:
table_rank_details

,Generalkey,TEAM_NAME,GAME_Date,GAME_Time,Comp,SEASON_Round,WEEK_DAY,TEAM_Venue,GAME_Result,GF,...,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Notes,Season
0,1,Sporting-KC,2020-02-29,19:30:00,MLS,Regular Season,Sat,Away,W,3,...,Vancouver,0.5,2.2,52,22120.0,Matt Besler,4-3-3,Ted Unkel,,2020
1,2,Sporting-KC,2020-03-07,19:30:00,MLS,Regular Season,Sat,Home,W,4,...,Houston Dynamo,2.6,0.7,56,21188.0,Matt Besler,4-3-3,Allen Chapman,,2020
2,3,Sporting-KC,2020-08-21,18:30:00,MLS,Regular Season,Fri,Away,W,2,...,Minnesota Utd,0.9,0.9,42,0.0,Alan Pulido,4-3-3,Ramy Touchan,,2020
3,4,Sporting-KC,2020-08-25,19:30:00,MLS,Regular Season,Tue,Home,L,2,...,Houston Dynamo,2.2,2.6,49,0.0,Alan Pulido,4-1-4-1,Joe Dickerson,,2020
4,5,Sporting-KC,2020-08-29,19:00:00,MLS,Regular Season,Sat,Away,D,1,...,Colorado Rapids,0.6,1.2,38,0.0,Alan Pulido,4-1-4-1,Armando Villarreal,,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,6086,Orlando-City,2018-10-06,19:00:00,MLS,Regular Season,Sat,Away,L,0,...,FC Dallas,0.5,0.8,48,16519.0,Sacha Kljestan,4-2-3-1,Nima Saghafi,,2018
6086,6087,Orlando-City,2018-10-13,19:30:00,MLS,Regular Season,Sat,Away,L,0,...,New England,0.5,1.6,40,19775.0,Lamine Sané,5-4-1,David Gantar,,2018
6087,6088,Orlando-City,2018-10-17,19:30:00,MLS,Regular Season,Wed,Home,L,1,...,Seattle,1.1,1.6,54,22344.0,Sacha Kljestan,4-2-3-1,Mark Geiger,,2018
6088,6089,Orlando-City,2018-10-21,15:00:00,MLS,Regular Season,Sun,Home,W,2,...,Columbus Crew,2.2,0.5,50,23642.0,Sacha Kljestan,3-4-3,Kevin Stott,,2018


# 8. Plots and Descriptive Analysis

# 9. Data Cleaning (removing duplicates)

# 10. Inserting cleaned tables to the Database